In [168]:
from bs4 import BeautifulSoup as bs
import urllib.request
import requests
import pprint
import pymongo 
import pprint

In [8]:
#Задаем желаемые языки программирования
profession = "programmer+"+(input("Пожалуйста, введите желаемый язык программирования на английском ").lower())
profession 

Пожалуйста, введите желаемый язык программирования на английском python


'programmer+python'

In [6]:
#Задаем колличество страниц поисковой выдачи. Отобразятся только те вакансии, в которых указана зарплата
page = int(input("Введите количество страниц, которое хотим спарсить "))

Введите количество страниц, которое хотим спарсить 2


In [10]:
#Задаем колличество вакансий на странице
items_on_page = int(input("Введите колличество вакансий на 1 странице - 20, 50 или 100 "))

Введите колличество вакансий на 1 странице - 20, 50 или 100 20


In [61]:
#Основной мега-цикл. 
#Отправляет настроенный линк, парсит полученную страницу, находит наименования вакансии, цену и ссылку.
#Повторяет столько раз, сколько страниц хочется спрарсить
vacancy_data = []
for i in range (0,page):
    link = f'https://hh.ru/search/vacancy?text={profession}&area=1&salary=&currency_code=RUR&only_with_salary=true&experience=doesNotMatter&order_by=relevance&search_period=&items_on_page={items_on_page}&no_magic=true&page={i}'
    html = urllib.request.urlopen(link)
    bytes_page = html.read()
    str_page = bytes_page.decode("utf8")
    html.close()
    parsed_html = bs(str_page,'lxml')
    EUR = 72 #курс евро
    USD = 64 #курс доллара
    for k in range (0,items_on_page):
        maincontent_name = parsed_html.findAll(attrs={"class": "bloko-link HH-LinkModifier"})[k]
        maincontent_money = parsed_html.findAll(attrs={"class": "vacancy-serp-item__compensation"})[k]
        vacancy_link = maincontent_name["href"]
        vacancy_name = maincontent_name.string
        vacancy_money_dirty = maincontent_money.string
        vacancy_money_clean = vacancy_money_dirty.replace("\xa0","").replace("от ","").replace("до","")
        if vacancy_money_clean.find('-')==-1:
            vacancy_money_min = vacancy_money_clean
            if vacancy_money_min.find("руб.")!=-1:
                vacancy_money_min=int(vacancy_money_min.replace("руб.",""))
            elif vacancy_money_min.find("руб.")==-1 and vacancy_money_min.find("EUR")!=-1:
                vacancy_money_min=vacancy_money_min.replace("EUR","")
                vacancy_money_min=vacancy_money_min*EUR
            elif vacancy_money_min.find("руб.")==-1 and vacancy_money_min.find("USD")!=-1:
                vacancy_money_min=vacancy_money_min.replace("USD","")
                vacancy_money_min=int(vacancy_money_min)*USD
            vacancy_money_max=vacancy_money_min
        else:
            vacancy_money_min = int(vacancy_money_clean[0: int(vacancy_money_clean.find('-'))])
            vacancy_money_max = vacancy_money_clean[int(vacancy_money_clean.find('-'))+1:]
            if vacancy_money_max.find("руб.")!=-1:
                vacancy_money_max=int(vacancy_money_max.replace("руб.",""))
            elif vacancy_money_max.find("руб.")==-1 and vacancy_money_max.find("EUR")!=-1:
                vacancy_money_max=int(vacancy_money_max.replace("EUR",""))
                vacancy_money_max=vacancy_money_max*EUR
                vacancy_money_min=vacancy_money_min*EUR
            elif vacancy_money_max.find("руб.")==-1 and vacancy_money_max.find("USD")!=-1:
                vacancy_money_max=int(vacancy_money_max.replace("USD",""))
                vacancy_money_max=vacancy_money_max*USD
                vacancy_money_min=vacancy_money_min*USD
        vacancy_data_page = [vacancy_name, vacancy_money_min,vacancy_money_max, vacancy_link]
        vacancy_data.append(vacancy_data_page)

[['Senior Front-end Developer (JavaScript, React)', 200000, 200000, 'https://hh.ru/vacancy/32144172?query=programmer%20python'], ['Front-end Developer (JavaScript, React)', 150000, 150000, 'https://hh.ru/vacancy/32077832?query=programmer%20python'], ['Data engineer (ML-разработчик, python)', 100000, 140000, 'https://hh.ru/vacancy/32099770?query=programmer%20python'], ['Программист Python', 80000, 80000, 'https://hh.ru/vacancy/32271659?query=programmer%20python'], ['Python Web Developer', 200000, 300000, 'https://hh.ru/vacancy/32096105?query=programmer%20python'], ['Junior Data Developer (Python)', 100000, 140000, 'https://hh.ru/vacancy/32030586?query=programmer%20python'], ['Senior Back-End Developer (PHP)', 210000, 250000, 'https://hh.ru/vacancy/32108917?query=programmer%20python'], ['Python Developer', 160000, 250000, 'https://hh.ru/vacancy/32072763?query=programmer%20python'], ['Программист Python', 60000, 100000, 'https://hh.ru/vacancy/32260756?query=programmer%20python'], ['Разраб

In [221]:
#Создаем Коллекцию вакансий в базе данных

client = MongoClient()
db = client.HH
vacancies = db.vacancies

In [222]:
#Вводим индекс - защиту от повторений
vacancies.create_index([('vacancy_id', pymongo.ASCENDING)], unique=True)

'vacancy_id_1'

In [223]:
#Создаем mongo-DB коллекцию с документами-вакансиями. id = id вакансии на сайте.
#Если вакансия уже есть - предупреждаем об этом 

for l in range (0,len(vacancy_data)):
    try:
        vacancy = {'vacancy_id': vacancy_data[l][3][22:vacancy_data[l][3].find("?")],
                   "vacancy_name":vacancy_data[l][0],
                      "vacancy_money_min":vacancy_data[l][1],
                      "vacancy_money_max":vacancy_data[l][2],
                      "vacancy_link":vacancy_data[l][3]}
        vacancies.insert_one(vacancy)
    except pymongo.errors.DuplicateKeyError:
        print ("Такая запись уже существует")

In [224]:
#Вакансия с максимальной зарплатой

max_money = vacancies.find().sort("vacancy_money_max",-1).limit(1)
for i in max_money:
    print("Вакансия с максимальной зарплатой",i)

Вакансия с максимальной зарплатой {'_id': ObjectId('5d25b942c2e33a8f5f8048e9'), 'vacancy_id': '32096105', 'vacancy_name': 'Python Web Developer', 'vacancy_money_min': 200000, 'vacancy_money_max': 300000, 'vacancy_link': 'https://hh.ru/vacancy/32096105?query=programmer%20python'}


In [225]:
#Вакансии с зарплатой выше вводимой

ambitions = int(input("Введите желаемую сумму в рублях. Вакансии с зарплатой ниже введенной отображаться не будут "))
greater_money = vacancies.find({"vacancy_money_max":{"$gt": ambitions}})
print(f"Вакансия с максимальной зарплатой выше {ambitions} рублей")
for i in greater_money:
    print(i)
    

Введите желаемую сумму в рублях. Вакансии с зарплатой ниже введенной отображаться не будут 200000
Вакансия с максимальной зарплатой выше 200000 рублей
{'_id': ObjectId('5d25b942c2e33a8f5f8048e9'), 'vacancy_id': '32096105', 'vacancy_name': 'Python Web Developer', 'vacancy_money_min': 200000, 'vacancy_money_max': 300000, 'vacancy_link': 'https://hh.ru/vacancy/32096105?query=programmer%20python'}
{'_id': ObjectId('5d25b942c2e33a8f5f8048eb'), 'vacancy_id': '32108917', 'vacancy_name': 'Senior Back-End Developer (PHP)', 'vacancy_money_min': 210000, 'vacancy_money_max': 250000, 'vacancy_link': 'https://hh.ru/vacancy/32108917?query=programmer%20python'}
{'_id': ObjectId('5d25b942c2e33a8f5f8048ec'), 'vacancy_id': '32072763', 'vacancy_name': 'Python Developer', 'vacancy_money_min': 160000, 'vacancy_money_max': 250000, 'vacancy_link': 'https://hh.ru/vacancy/32072763?query=programmer%20python'}
{'_id': ObjectId('5d25b942c2e33a8f5f8048ef'), 'vacancy_id': '31844761', 'vacancy_name': 'Разработчик Pyt